In [1]:
import sys
import os
from icecream import ic

from pathlib import Path

import utils_behavior

from utils_behavior import Ballpushing_utils
from utils_behavior import Utils
from utils_behavior import Processing
from utils_behavior import HoloviewsTemplates

import pandas as pd
import hvplot.pandas
import numpy as np


from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd
import pyarrow.feather as feather

import pandas as pd
import numpy as np
import holoviews as hv
import hvplot.pandas



import matplotlib.pyplot as plt
import seaborn as sns



import holoviews as hv


hv.extension("bokeh")

In [2]:
# Get the path to the data

Datapath = Utils.get_data_path()

In [3]:
# Find folders with "Learning or learning" in the name as a list

folders = [f for f in Datapath.glob("*FeedingState*")]

folders

[PosixPath('/mnt/upramdya_data/MD/MultiMazeRecorder/Videos/230704_FeedingState_1_AM_Videos_Tracked'),
 PosixPath('/mnt/upramdya_data/MD/MultiMazeRecorder/Videos/240718_Afternoon_FeedingState_Videos_Tracked'),
 PosixPath('/mnt/upramdya_data/MD/MultiMazeRecorder/Videos/230704_FeedingState_1_PM_Videos_Tracked'),
 PosixPath('/mnt/upramdya_data/MD/MultiMazeRecorder/Videos/240718_Afternoon_FeedingState_next_Videos_Tracked'),
 PosixPath('/mnt/upramdya_data/MD/MultiMazeRecorder/Videos/230705_FeedingState_2_AM_Videos_Tracked'),
 PosixPath('/mnt/upramdya_data/MD/MultiMazeRecorder/Videos/230706_FeedingState_3_PM_Flipped_Videos_Tracked'),
 PosixPath('/mnt/upramdya_data/MD/MultiMazeRecorder/Videos/230725_FeedingState_PM_Dark_Videos_Tracked'),
 PosixPath('/mnt/upramdya_data/MD/MultiMazeRecorder/Videos/230727_FeedingState_PM_Dark_Flip_Videos_Tracked'),
 PosixPath('/mnt/upramdya_data/MD/MultiMazeRecorder/Videos/230705_FeedingState_2_PM_Videos_Tracked')]

In [4]:
# Take the first 

exp = Ballpushing_utils.Experiment(folders[0])

Loading fly from /mnt/upramdya_data/MD/MultiMazeRecorder/Videos/230704_FeedingState_1_AM_Videos_Tracked/arena2/corridor5
Loading fly from /mnt/upramdya_data/MD/MultiMazeRecorder/Videos/230704_FeedingState_1_AM_Videos_Tracked/arena2/corridor1
Loading fly from /mnt/upramdya_data/MD/MultiMazeRecorder/Videos/230704_FeedingState_1_AM_Videos_Tracked/arena2/corridor4
Loading fly from /mnt/upramdya_data/MD/MultiMazeRecorder/Videos/230704_FeedingState_1_AM_Videos_Tracked/arena2/corridor2
Loading fly from /mnt/upramdya_data/MD/MultiMazeRecorder/Videos/230704_FeedingState_1_AM_Videos_Tracked/arena2/corridor6
Loading fly from /mnt/upramdya_data/MD/MultiMazeRecorder/Videos/230704_FeedingState_1_AM_Videos_Tracked/arena2/corridor3
Loading fly from /mnt/upramdya_data/MD/MultiMazeRecorder/Videos/230704_FeedingState_1_AM_Videos_Tracked/arena7/corridor5
Loading fly from /mnt/upramdya_data/MD/MultiMazeRecorder/Videos/230704_FeedingState_1_AM_Videos_Tracked/arena7/corridor1
Loading fly from /mnt/upramdya_d

In [5]:
Dataset = Ballpushing_utils.Dataset(exp)

In [6]:
Dataset.generate_dataset(success_cutoff=False)

,yball,xball,yfly,xfly,Frame,time,yball_smooth,xball_smooth,yfly_smooth,xfly_smooth,...,Nickname,Brain region,Date,Genotype,Period,FeedingState,Orientation,Light,SampleSize,label
0,331.699677,46.302090,484.889160,43.974110,1,0.033333,331.692498,46.413479,484.955528,44.054332,...,PR,Control,230704,PR,AM,starved_noWater,std,on,53,PR (n = 53)
1,331.726562,46.413559,484.851685,44.008175,2,0.066667,331.692508,46.413505,484.955307,44.054328,...,PR,Control,230704,PR,AM,starved_noWater,std,on,53,PR (n = 53)
2,331.732147,46.455559,484.923004,44.029278,3,0.100000,331.692519,46.413530,484.955085,44.054324,...,PR,Control,230704,PR,AM,starved_noWater,std,on,53,PR (n = 53)
3,331.710449,46.429592,484.933014,44.094261,4,0.133333,331.692530,46.413556,484.954863,44.054320,...,PR,Control,230704,PR,AM,starved_noWater,std,on,53,PR (n = 53)
4,331.725586,46.419800,484.881714,43.968246,5,0.166667,331.692540,46.413582,484.954642,44.054316,...,PR,Control,230704,PR,AM,starved_noWater,std,on,53,PR (n = 53)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11447995,278.370392,42.644981,477.764008,50.702675,215996,7199.866667,278.376881,42.587924,483.560921,52.246405,...,PR,Control,230704,PR,AM,Fed,std,on,53,PR (n = 53)
11447996,278.384857,42.590366,476.146332,50.931297,215997,7199.900000,278.376993,42.587996,483.795246,52.271270,...,PR,Control,230704,PR,AM,Fed,std,on,53,PR (n = 53)
11447997,278.346436,42.608295,479.940704,55.232399,215998,7199.933333,278.377106,42.588069,484.029571,52.296136,...,PR,Control,230704,PR,AM,Fed,std,on,53,PR (n = 53)
11447998,278.336487,42.620457,478.189667,50.862263,215999,7199.966667,278.377218,42.588141,484.263897,52.321002,...,PR,Control,230704,PR,AM,Fed,std,on,53,PR (n = 53)


In [7]:
Dataset.data.columns

Index(['yball', 'xball', 'yfly', 'xfly', 'Frame', 'time', 'yball_smooth',
       'xball_smooth', 'yfly_smooth', 'xfly_smooth', 'yfly_relative',
       'yball_relative', 'event', 'fly', 'flypath', 'experiment', 'Nickname',
       'Brain region', 'Date', 'Genotype', 'Period', 'FeedingState',
       'Orientation', 'Light', 'SampleSize', 'label'],
      dtype='object')

In [15]:
# Plot the yball_relative as a function of time grouped by individual flies

# curves = hv.Curve(Dataset.data, kdims=["time"], vdims=["yball_relative","fly"]).groupby("fly").overlay()

# curves.opts(width=800, height=400)

In [18]:
from holoviews import opts
from bokeh.palettes import viridis

Tries = 30

NbFlies = 7

for i in range(Tries):

    # Pick random flies and plot their yball_relative as a function of time
    flies = np.random.choice(Dataset.data.fly.unique(), NbFlies)

    # Generate a Viridis palette with adapted number of colors colors
    viridis_palette = viridis(NbFlies)

    # Create the curves and overlay them
    curves = (
        hv.Curve(
            Dataset.data[Dataset.data.fly.isin(flies)],
            kdims=["time"],
            vdims=["yball_relative", "fly"],
        )
        .groupby("fly")
        .overlay()
    )

    curves = curves.opts(
        opts.Curve(
            width=800,
            height=400,
            show_grid=True,
            show_legend=False,
            ylabel="ball position relative to start (px)",
            xlabel="time (s)",
            
        )
    )

    # Apply the viridis colormap
    curves = curves.opts(opts.Curve(color=hv.Cycle(values=list(viridis_palette))))

    hv.save(curves, f"/mnt/upramdya_files/DURRIEU_Matthias/DlsImgs/testcurves{i}.png")